# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [17]:
import json
import os
import logging
from typing import Sequence, List

import openai
from dotenv import load_dotenv
from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)
logger.setLevel(logging.DEBUG)
logging.getLogger("llama_index").setLevel(logging.DEBUG)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

load_dotenv("../../../.env")
assert (key := os.getenv("OPENAI_API_KEY")); openai.api_key = key
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [18]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [19]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [20]:
from llama_index.agent import OpenAIAgent
from llama_index.agent.experimental_openai_agent import (
    ExperimentalOpenAIAgent
)
from llama_index.llms import OpenAI

In [21]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
exp_agent = ExperimentalOpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True
)

In [22]:
#print(agent.chat("Tell me a random name"))

In [7]:
#print(agent.chat("Now tell me that name backwards."))

In [9]:
#response = exp_agent.chat("What is (121 * 3) + 42?")

### Chat

In [22]:
response = exp_agent.chat("What is (121 * 3) + 42?")
print(str(response))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "What is (121 * 3) + 42?"}], "stream": false, "model": "gpt-3.5-turbo-0613", "temperature": 0.0, "max_tokens": null, "functions": [{"name": "multiply", "description": "multiply(a: int, b: int) -> int\\nMultiple two integers and returns the result integer", "parameters": {"title": "multiply", "type": "object", "properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"]}}, {"name": "add", "description": "add(a: int, b: int) -> int\\nAdd two integers and returns the result integer", "parameters": {"title": "add", "type": "object", "properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"]}}]}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=940 request_id=9013bf71b78816bc8c96deace484333a response_code=200
DEBUG:llama_index.agent.experimental_openai_agent:in _call_function
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "What is (121 * 3) + 42?"}, {"role": "assistant", "content": null, "function_call": {"name": "multiply", "arguments": "{\\n  \\"a\\": 121,\\n  \\"b\\": 3\\n}"}}, {"role": "function", "content": "363", "name": "multiply"}, {"role": "assistant", "content": null, "function_call": {"name": "add", "arguments": "{\\n  \\"a\\": 363,\\n  \\"b\\": 42\\n}"}}, {"role": "function", "content": "405", "name": "add"}], "stream": false, "model": "gpt-3.5-turbo-0613", "temperature": 0.0, 

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405


KeyboardInterrupt: 

In [10]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [23]:
response = await exp_agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [6]:
from llama_index.agent.experimental_openai_agent import logger as _logger
assert logging.getLogger("llama_index.agent.experimental_openai_agent") is _logger

In [15]:
_logger.exception("wqwloidjqojd")

In [8]:
logging.INFO

20

In [7]:
_logger.level

10

In [9]:
response = exp_agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a limerick about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242
There once was a group of mice,
Whose number was quite precise.
Two hundred forty-two,
They scurried and they flew,
In search of cheese, their ultimate prize.

### Async Streaming Chat

In [24]:
logging.getLogger("asyncio").setLevel(logging.DEBUG)

In [7]:
response = await exp_agent.astream_chat(
    "What is 121 * 2 + 8? Once you have the answer, use that number to write a limerick about a group of mice."
)
print("resp")
response_gen = response.response_gen
print("gen", response_gen)
raise Exception
for token in response_gen:
    print(token, end="")

resp
gen <generator object StreamingAgentChatResponse.response_gen at 0x7f2ae34c3ed0>


Exception: 

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)